# Segger Minimal Example

In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
from segger.data.io import XeniumSample
from segger.training.train import LitSegger
from segger.training.segger_data_module import SeggerDataModule
from pytorch_lightning import Trainer
from pathlib import Path
import warnings

## Create Dataset

Explain things here

In [4]:
xenium_data_dir = Path('../../dev/tutorial/xenium_data/')
segger_data_dir = Path('../../dev/tutorial/segger_data/')

In [6]:
xs = XeniumSample(verbose=True)
xs.set_file_paths(
    transcripts_path=xenium_data_dir / 'transcripts.parquet',
    boundaries_path=xenium_data_dir / 'nucleus_boundaries.parquet',
)

In [7]:
xs.save_dataset_for_segger(
    processed_dir=segger_data_dir,
    x_size=200,
    y_size=200,
    d_x=180,
    d_y=180,
    margin_x=10,
    margin_y=10,
)

[########################################] | 100% Completed | 57.36 s


## Train Segger Model

Explain things here

In [27]:
from pytorch_lightning.plugins.environments import SLURMEnvironment
SLURMEnvironment.detect = lambda: False

In [11]:
models_dir = Path('../../dev/tutorial/models/')

In [30]:
# Initialize the Lightning model
metadata = (["tx", "nc"], [("tx", "belongs", "nc"), ("tx", "neighbors", "tx")])
ls = LitSegger(
    num_tx_tokens=500,
    init_emb=8,
    hidden_channels=64,
    out_channels=16,
    heads=4,
    num_mid_layers=1,
    aggr='sum',
    metadata=metadata,
)

In [34]:
# Initialize the Lightning data module
dm = SeggerDataModule(
    data_dir=segger_data_dir,
    batch_size=4,  # explain other options above
    num_workers=4,  # explain other options above
)

In [31]:
# Initialize the Lightning trainer
trainer = Trainer(
    accelerator='cuda',  # explain other options above
    strategy='auto',
    precision='16-mixed',
    devices=1,  # explain other options above
    max_epochs=100,  # explain other options above
    default_root_dir=models_dir,
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [38]:
import torch

In [40]:
# Fit model
trainer.fit(model=ls, datamodule=dm)

You are using a CUDA device ('NVIDIA A100 80GB PCIe MIG 2g.20gb') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


FileNotFoundError: [Errno 2] No such file or directory: '../../dev/tutorial/segger_data/train_tiles/raw'